In [ ]:
import sys
sys.path.insert(0,'/home/pi/zumi/lib')
import Speaker as speaker
import Engine as engine
from IPython import display 
import numpy as np
import PIL.Image 
from TawnCam import PiCamera
import time
import cv2

import TouristDemoHelper as tourist_demo

model = tourist_demo.load_model("landmark")
# speaker.play()


In [ ]:
import TouristDemoHelper as tourist_demo

camera = PiCamera(image_w=64, image_h=64, image_d=3, framerate=10)

try:
    while True:

        frame = camera.run()
        frame = cv2.flip(frame, -1)
        
        predictions = model.predict(frame[None, :, :, :])
        index_of_most_likely_prediction = np.argmax(predictions[0])
        predicted_label = tourist_demo.landmarks.get(index_of_most_likely_prediction)
                          
        display.clear_output(wait=True)
        display.display(PIL.Image.fromarray(frame))
        display.display(predicted_label)
        display.display(tourist_demo.get_readable_predictions(predictions))
        
        is_correct = input("Input 'n' if incorrect: ") 
        if is_correct == 'n':
            correct_label = input("what is the correct label? ")
            tourist_demo.take_a_bunch_of_pictures(camera, correct_label)
        else:
            correct_label = predicted_label
                                       
        if correct_label == "up":
            engine.forward_a_bit()
        elif correct_label == 'left':
            engine.left_a_bit()
        elif correct_label == "right":
            engine.right_a_bit()
        else:
            engine.forward_a_bit()
        
        engine.stop()
    
finally:
    engine.stop()
    camera.shutdown()